In [2]:
import os
os.environ["TORCH_HOME"] = r"O:\O drive\AI\my project\medical image projects\breast_termo"

import torch
print(torch.hub.get_dir())
print(os.environ["TORCH_HOME"])


O:\O drive\AI\my project\medical image projects\breast_termo\hub
O:\O drive\AI\my project\medical image projects\breast_termo


In [2]:

torch.cuda.empty_cache()

os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

NameError: name 'torch' is not defined

In [3]:
import os
import cv2
import numpy as np
import pandas as pd
from PIL import Image
import random
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

# Set random seed for reproducibility
random.seed(42)
np.random.seed(42)

def create_dataframes(root_dir, train_split=0.7, val_split=0.15):
    """
    Create DataFrames for train, validation, and test splits from a single directory with class folders.

    Directory structure:
    <root_dir>/
        normal/
        Sick/
        Unknown_class/

    Args:
        root_dir (str): Root directory of the dataset.
        train_split (float): Proportion of data for training (default: 0.7).
        val_split (float): Proportion of data for validation (default: 0.15).

    Returns:
        tuple: (train_df, val_df, test_df, class_to_idx)
            - train_df, val_df, test_df: DataFrames with columns "file_path", "label", "class_name"
            - class_to_idx: Dict mapping class names to numeric indices
    """
    valid_extensions = {'.png', '.jpg', '.jpeg', '.tif', '.bmp'}
    data = []
    class_names = ['normal', 'Sick', 'Unknown_class']
    class_to_idx = {name: idx for idx, name in enumerate(class_names)}

    # Collect all image paths and labels
    for class_name in class_names:
        class_dir = os.path.join(root_dir, class_name)
        if not os.path.isdir(class_dir):
            print(f"Warning: {class_dir} does not exist!")
            continue

        label = class_to_idx[class_name]
        for file in os.listdir(class_dir):
            if os.path.splitext(file)[1].lower() in valid_extensions:
                image_path = os.path.join(class_dir, file)
                data.append({
                    "file_path": image_path,
                    "label": label,
                    "class_name": class_name
                })

    # Create a single DataFrame
    full_df = pd.DataFrame(data)
    
    if full_df.empty:
        print("Error: No valid images found in the dataset!")
        return pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), class_to_idx

    # Split the data into train, validation, and test sets
    train_df, temp_df = train_test_split(
        full_df, 
        train_size=train_split, 
        stratify=full_df['label'], 
        random_state=42
    )
    val_size = val_split / (1 - train_split)  # Adjust validation size for remaining data
    val_df, test_df = train_test_split(
        temp_df, 
        train_size=val_size, 
        stratify=temp_df['label'], 
        random_state=42
    )

    # Reset indices for cleanliness
    train_df = train_df.reset_index(drop=True)
    val_df = val_df.reset_index(drop=True)
    test_df = test_df.reset_index(drop=True)

    # Print basic statistics
    for df, split in zip([train_df, val_df, test_df], ["Train", "Validation", "Test"]):
        if df.empty:
            print(f"Warning: No images in {split} split!")
        else:
            print(f"\n{split} Dataset Statistics:")
            print(f"Total images: {len(df)}")
            print(f"Class distribution:\n{df['class_name'].value_counts()}")

    return train_df, val_df, test_df, class_to_idx

def analyze_and_plot_dataframes(train_df, val_df, test_df, class_to_idx, save_dir):
    """
    Analyze and visualize each DataFrame.

    Args:
        train_df, val_df, test_df (pd.DataFrame): DataFrames for each split.
        class_to_idx (dict): Mapping of class names to indices.
        save_dir (str): Directory to save plots.
    """
    os.makedirs(save_dir, exist_ok=True)

    # Basic statistics and duplicates
    for df, split in zip([train_df, val_df, test_df], ["Train", "Validation", "Test"]):
        if not df.empty:
            print(f"\n{split} Dataset Statistics:")
            print(f"Total images: {len(df)}")
            print(f"Class distribution:\n{df['class_name'].value_counts()}")
            print(f"Duplicate images: {df['file_path'].duplicated().sum()}")

    # Dynamic color palette for classes
    class_names = sorted(class_to_idx.keys())
    colors = sns.color_palette("husl", len(class_names))
    color_map = {idx: colors[i] for i, idx in enumerate(class_to_idx.values())}

    # Plot class distribution for each split
    for df, split in zip([train_df, val_df, test_df], ["Train", "Validation", "Test"]):
        if df.empty:
            continue
        plt.figure(figsize=(10, 6))
        counts = df['label'].value_counts().sort_index()
        bars = plt.bar(counts.index, counts.values, color=[color_map[i] for i in counts.index])
        
        plt.ylim(0, counts.max() + counts.max() * 0.1)
        for bar in bars:
            height = bar.get_height()
            plt.annotate(f'{int(height)}',
                         xy=(bar.get_x() + bar.get_width() / 2, height),
                         xytext=(0, 5),
                         textcoords='offset points',
                         ha='center', va='bottom')
        
        plt.xlabel('Class')
        plt.ylabel('Count')
        plt.title(f'Class Distribution in {split} Set')
        plt.xticks(counts.index, [class_names[i] for i in counts.index], rotation=45)
        plt.grid(axis='y', alpha=0.7)
        plt.tight_layout()
        plt.savefig(os.path.join(save_dir, f"{split.lower()}_class_distribution.png"))
        plt.close()

    # Plot random samples (up to 10 per class) for each split
    for df, split in zip([train_df, val_df, test_df], ["Train", "Validation", "Test"]):
        if df.empty:
            continue
        classes = sorted(df['class_name'].unique())
        samples_per_class = min(10, df['class_name'].value_counts().min())
        
        # Calculate grid size
        n_cols = min(5, samples_per_class)
        n_rows = (samples_per_class + n_cols - 1) // n_cols * len(classes)
        
        plt.figure(figsize=(n_cols * 4, n_rows * 4))
        plot_idx = 1
        
        for class_name in classes:
            class_images = df[df['class_name'] == class_name]['file_path'].values
            if len(class_images) == 0:
                continue
            samples = np.random.choice(class_images, min(samples_per_class, len(class_images)), replace=False)
            
            for img_path in samples:
                plt.subplot(n_rows, n_cols, plot_idx)
                img = Image.open(img_path)
                # Convert grayscale to RGB if necessary
                if img.mode != 'RGB':
                    img = img.convert('RGB')
                plt.imshow(img)
                plt.title(class_name, fontsize=12)
                plt.axis('off')
                plot_idx += 1
        
        plt.tight_layout()
        plt.savefig(os.path.join(save_dir, f"{split.lower()}_random_samples.png"))
        plt.close()

# Example usage
if __name__ == "__main__":
    root_dir = r"O:\O drive\AI\my project\medical image projects\breast_termo\archive\BCD_Dataset"
    save_dir = r"O:\O drive\AI\my project\medical image projects\breast_termo\plots"
    train_df, val_df, test_df, class_to_idx = create_dataframes(root_dir)
    analyze_and_plot_dataframes(train_df, val_df, test_df, class_to_idx, save_dir)


Train Dataset Statistics:
Total images: 175
Class distribution:
class_name
Sick             70
Unknown_class    70
normal           35
Name: count, dtype: int64

Validation Dataset Statistics:
Total images: 37
Class distribution:
class_name
Unknown_class    15
Sick             14
normal            8
Name: count, dtype: int64

Test Dataset Statistics:
Total images: 39
Class distribution:
class_name
Sick             16
Unknown_class    15
normal            8
Name: count, dtype: int64

Train Dataset Statistics:
Total images: 175
Class distribution:
class_name
Sick             70
Unknown_class    70
normal           35
Name: count, dtype: int64
Duplicate images: 0

Validation Dataset Statistics:
Total images: 37
Class distribution:
class_name
Unknown_class    15
Sick             14
normal            8
Name: count, dtype: int64
Duplicate images: 0

Test Dataset Statistics:
Total images: 39
Class distribution:
class_name
Sick             16
Unknown_class    15
normal            8
Name: coun

In [4]:
import matplotlib.pyplot as plt
import seaborn as sns
import os
import pandas as pd
from sklearn.utils import resample

# Pie Chart 1: Dataset Split Distribution
def plot_dataset_split_pie(train_df, val_df, test_df, save_dir="data_analysis"):
    """
    Plot a pie chart showing the distribution of samples across train, val, and test splits.

    Args:
        train_df, val_df, test_df (pd.DataFrame): DataFrames for each split.
        save_dir (str): Directory to save the plot.
    """
    sizes = [len(train_df), len(val_df), len(test_df)]
    labels = ['Training', 'Validation', 'Test']
    total = sum(sizes)
    colors = sns.color_palette("husl", 3)  # Consistent palette for 3 classes

    fig, ax = plt.subplots(figsize=(8, 8))
    wedges, texts, autotexts = ax.pie(
        sizes,
        labels=labels,
        autopct=lambda pct: f"{pct:.1f}%\n({int(pct/100.*total)})",
        startangle=140,
        colors=colors,
        shadow=True,
        wedgeprops={'edgecolor': 'white', 'linewidth': 1}
    )

    for text in texts:
        text.set_fontsize(14)
        text.set_fontweight('bold')
    for autotext in autotexts:
        autotext.set_fontsize(12)

    ax.set_title("Breast Cancer Thermography Dataset Split Distribution", fontsize=16, fontweight='bold')
    plt.text(0, -1.3, f"Total Samples: {total}", fontsize=12, ha='center', style='italic')
    plt.tight_layout()
    
    os.makedirs(save_dir, exist_ok=True)
    plt.savefig(os.path.join(save_dir, "dataset_split_pie.png"))
    plt.close()

# Pie Chart 2: Test Set Class Distribution
def plot_test_class_pie(test_df, class_to_idx, save_dir="data_analysis"):
    """
    Plot a pie chart showing the class distribution in the test set.

    Args:
        test_df (pd.DataFrame): Test DataFrame with 'label' and 'class_name' columns.
        class_to_idx (dict): Mapping of class names to numeric indices.
        save_dir (str): Directory to save the plot.
    """
    if test_df.empty:
        print("Warning: Test DataFrame is empty, skipping test class pie chart.")
        return

    class_counts = test_df['label'].value_counts().sort_index()
    sizes = class_counts.values
    labels = [list(class_to_idx.keys())[list(class_to_idx.values()).index(i)] for i in class_counts.index]
    total = sum(sizes)
    colors = sns.color_palette("husl", len(labels))  # Consistent palette

    fig, ax = plt.subplots(figsize=(8, 8))
    wedges, texts, autotexts = ax.pie(
        sizes,
        labels=labels,
        autopct=lambda pct: f"{pct:.1f}%\n({int(pct/100.*total)})",
        startangle=140,
        colors=colors,
        shadow=True,
        wedgeprops={'edgecolor': 'white', 'linewidth': 1}
    )

    for text in texts:
        text.set_fontsize(14)
        text.set_fontweight('bold')
    for autotext in autotexts:
        autotext.set_fontsize(12)

    ax.set_title("Test Set Class Distribution (Normal, Sick, Unknown)", fontsize=16, fontweight='bold')
    plt.text(0, -1.3, f"Total Test Samples: {total}", fontsize=12, ha='center', style='italic')
    plt.tight_layout()
    
    os.makedirs(save_dir, exist_ok=True)
    plt.savefig(os.path.join(save_dir, "test_class_pie.png"))
    plt.close()

# Pie Chart 3: Balanced Training Set Class Distribution
def plot_train_balanced_class_pie(train_df_balanced, class_to_idx, save_dir="data_analysis"):
    """
    Plot a pie chart showing the class distribution in the balanced training set.

    Args:
        train_df_balanced (pd.DataFrame): Balanced training DataFrame with 'label' and 'class_name' columns.
        class_to_idx (dict): Mapping of class names to numeric indices.
        save_dir (str): Directory to save the plot.
    """
    if train_df_balanced.empty:
        print("Warning: Balanced Train DataFrame is empty, skipping balanced train class pie chart.")
        return

    class_counts = train_df_balanced['label'].value_counts().sort_index()
    sizes = class_counts.values
    labels = [list(class_to_idx.keys())[list(class_to_idx.values()).index(i)] for i in class_counts.index]
    total = sum(sizes)
    colors = sns.color_palette("husl", len(labels))  # Consistent palette

    fig, ax = plt.subplots(figsize=(8, 8))
    wedges, texts, autotexts = ax.pie(
        sizes,
        labels=labels,
        autopct=lambda pct: f"{pct:.1f}%\n({int(pct/100.*total)})",
        startangle=140,
        colors=colors,
        shadow=True,
        wedgeprops={'edgecolor': 'white', 'linewidth': 1}
    )

    for text in texts:
        text.set_fontsize(14)
        text.set_fontweight('bold')
    for autotext in autotexts:
        autotext.set_fontsize(12)

    ax.set_title("Balanced Training Set Class Distribution (Normal, Sick, Unknown)", fontsize=16, fontweight='bold')
    plt.text(0, -1.3, f"Total Balanced Train Samples: {total}", fontsize=12, ha='center', style='italic')
    plt.tight_layout()
    
    os.makedirs(save_dir, exist_ok=True)
    plt.savefig(os.path.join(save_dir, "train_balanced_class_pie.png"))
    plt.close()

# Resample train_df to balance classes
def resample_train_df(train_df, class_to_idx, save_dir="data_analysis"):
    """
    Resample the training DataFrame to balance classes by oversampling minority classes.

    Args:
        train_df (pd.DataFrame): Training DataFrame with 'file_path', 'label', 'class_name' columns.
        class_to_idx (dict): Mapping of class names to numeric indices.
        save_dir (str): Directory to save the distribution plot.

    Returns:
        pd.DataFrame: Balanced training DataFrame.
    """
    if train_df.empty:
        print("Error: Training DataFrame is empty, cannot balance.")
        return train_df

    # Get class counts
    class_counts = train_df['label'].value_counts()
    majority_count = class_counts.max()
    majority_label = class_counts.idxmax()

    # Separate DataFrames for each class
    dfs_by_class = [train_df[train_df['label'] == label] for label in class_counts.index]

    # Oversample minority classes to match majority
    balanced_dfs = []
    for df_class, label in zip(dfs_by_class, class_counts.index):
        if label == majority_label:
            balanced_dfs.append(df_class)
        else:
            df_oversampled = resample(
                df_class,
                replace=True,
                n_samples=majority_count,
                random_state=42
            )
            balanced_dfs.append(df_oversampled)

    # Combine all classes
    train_df_balanced = pd.concat(balanced_dfs)

    # Shuffle the dataset
    train_df_balanced = train_df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

    # Print new class distribution
    print("\nNew class distribution after oversampling:")
    print(train_df_balanced['label'].value_counts())

    # Visualize new class distribution (bar plot)
    class_names = sorted(class_to_idx, key=class_to_idx.get)  # Sort by index
    colors = sns.color_palette("husl", len(class_names))
    color_map = {i: colors[i] for i in range(len(class_names))}

    plt.figure(figsize=(10, 6))
    counts = train_df_balanced['label'].value_counts().sort_index()
    bars = plt.bar(counts.index, counts.values, color=[color_map[i] for i in counts.index])

    plt.ylim(0, counts.max() + counts.max() * 0.1)
    for bar in bars:
        height = bar.get_height()
        plt.annotate(f'{int(height)}',
                     xy=(bar.get_x() + bar.get_width() / 2, height),
                     xytext=(0, 5),
                     textcoords='offset points',
                     ha='center', va='bottom')

    plt.xlabel('Class')
    plt.ylabel('Count')
    plt.title('Class Distribution in Balanced Training Set (Normal, Sick, Unknown)')
    plt.xticks(counts.index, [class_names[i] for i in counts.index], rotation=45)
    plt.grid(axis='y', alpha=0.7)
    plt.tight_layout()
    os.makedirs(save_dir, exist_ok=True)
    plt.savefig(os.path.join(save_dir, "train_balanced_class_bar.png"))
    plt.close()

    # Plot pie chart for balanced training set
    plot_train_balanced_class_pie(train_df_balanced, class_to_idx, save_dir)

    return train_df_balanced

# Example usage (assuming train_df, val_df, test_df, class_to_idx are from previous code)
if __name__ == "__main__":
    # Placeholder for DataFrames (replace with actual DataFrames from previous code)
    root_dir = r"O:\O drive\AI\my project\medical image projects\breast_termo\archive\BCD_Dataset"
    save_dir = r"O:\O drive\AI\my project\medical image projects\breast_termo\plots"
    
    # Assuming create_dataframes is defined as in the previous artifact
    
    train_df, val_df, test_df, class_to_idx = create_dataframes(root_dir)
    
    # Plot dataset split and test class distribution
    plot_dataset_split_pie(train_df, val_df, test_df, save_dir)
    plot_test_class_pie(test_df, class_to_idx, save_dir)
    
    # Balance the training set and visualize
    train_df_balanced = resample_train_df(train_df, class_to_idx, save_dir)


Train Dataset Statistics:
Total images: 175
Class distribution:
class_name
Sick             70
Unknown_class    70
normal           35
Name: count, dtype: int64

Validation Dataset Statistics:
Total images: 37
Class distribution:
class_name
Unknown_class    15
Sick             14
normal            8
Name: count, dtype: int64

Test Dataset Statistics:
Total images: 39
Class distribution:
class_name
Sick             16
Unknown_class    15
normal            8
Name: count, dtype: int64

New class distribution after oversampling:
label
1    70
0    70
2    70
Name: count, dtype: int64


In [5]:
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
from PIL import Image, ImageFile
from sklearn.utils.class_weight import compute_class_weight
import albumentations as A
from albumentations.pytorch import ToTensorV2
import os
import gc
import logging

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Handle truncated/corrupted images
ImageFile.LOAD_TRUNCATED_IMAGES = True

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
logger.info(f"Using device: {device}")

# Define preprocessing transforms with albumentations
def get_transforms(split, mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)):
    """
    Define Albumentations transforms for train/val/test splits, tailored for thermography images.

    Args:
        split (str): Dataset split ('train', 'val', or 'test').
        mean (tuple): Mean values for normalization (default: ImageNet means).
        std (tuple): Standard deviation values for normalization (default: ImageNet stds).

    Returns:
        A.Compose: Albumentations transformation pipeline.
    """
    if split == "train":
        return A.Compose([
            A.Resize(224, 224, always_apply=True),
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.5),
            A.Rotate(limit=30, p=0.5),  # Reduced rotation for thermography images
            A.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.0, p=0.5),  # No hue shift for thermal images
            A.CLAHE(clip_limit=4.0, tile_grid_size=(8, 8), p=0.5),  # Enhance contrast for thermography
            A.RandomBrightnessContrast(brightness_limit=0.1, contrast_limit=0.1, p=0.5),
            A.GaussianBlur(blur_limit=(3, 5), p=0.3),  # Reduced blur intensity
            A.Normalize(mean=mean, std=std),
            ToTensorV2(),
        ])
    else:
        return A.Compose([
            A.Resize(224, 224, always_apply=True),
            A.Normalize(mean=mean, std=std),
            ToTensorV2(),
        ])

# Wrapper to use albumentations with PyTorch
class AlbumentationsTransform:
    def __init__(self, transform):
        self.transform = transform

    def __call__(self, img):
        img = np.array(img)
        augmented = self.transform(image=img)
        return augmented['image']

# Custom Dataset for DataFrame
class DataFrameDataset(Dataset):
    def __init__(self, df, transform=None):
        """
        Initialize dataset from DataFrame.

        Args:
            df (pd.DataFrame): DataFrame with 'file_path', 'label', 'class_name' columns.
            transform: Albumentations transformation pipeline.
        """
        self.df = df.reset_index(drop=True)
        self.transform = transform
        self.classes = sorted(self.df['class_name'].unique())
        self.class_to_idx = {cls: idx for idx, cls in enumerate(self.classes)}
        self.df = self._validate_and_filter_dataset()
        unique_labels = sorted(self.df['label'].unique())
        expected_labels = list(range(len(self.classes)))
        if sorted(unique_labels) != expected_labels:
            logger.error(f"Label mismatch: Expected labels {expected_labels}, found {unique_labels}")
            raise ValueError(f"Label mismatch: Expected {expected_labels}, found {unique_labels}")
        logger.info(f"Dataset size after filtering: {len(self.df)}")
        logger.info(f"Classes: {self.classes}")
        logger.info(f"Class to index: {self.class_to_idx}")
        logger.info(f"Class distribution:\n{self.df['class_name'].value_counts()}")

    def _validate_and_filter_dataset(self):
        """
        Filter out invalid images or labels from the dataset.

        Returns:
            pd.DataFrame: Filtered DataFrame.
        """
        valid_rows = []
        for idx in range(len(self.df)):
            row = self.df.iloc[idx]
            img_path = row['file_path']
            label = row['label']
            if not os.path.isfile(img_path):
                logger.warning(f"Invalid file path at index {idx}: {img_path}")
                continue
            if not isinstance(label, (int, np.integer)) or label not in range(len(self.classes)):
                logger.warning(f"Invalid label at index {idx}: {label}")
                continue
            try:
                img = Image.open(img_path).convert('RGB')
                img.close()
            except Exception as e:
                logger.warning(f"Corrupted image at index {idx}: {img_path}, error: {e}")
                continue
            valid_rows.append(idx)
        if not valid_rows:
            raise ValueError("No valid items in dataset after filtering")
        filtered_df = self.df.iloc[valid_rows].reset_index(drop=True)
        logger.info(f"Filtered out {len(self.df) - len(filtered_df)} invalid rows")
        return filtered_df

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = row['file_path']
        label = row['label']
        try:
            img = Image.open(img_path).convert('RGB')  # Ensure RGB for thermography images
        except Exception as e:
            logger.error(f"Error loading image {img_path}: {e}")
            return None
        if self.transform:
            img = self.transform(img)
        label_tensor = torch.tensor(label, dtype=torch.long)
        if label_tensor.numel() != 1:
            logger.error(f"Label at index {idx} is not scalar: {label_tensor}")
            return None
        logger.debug(f"Item {idx}: label {label} -> tensor {label_tensor}")
        return img, label_tensor

# Custom Dataset Wrapper
class CustomDataset(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        item = self.dataset[idx]
        if item is None:
            logger.warning(f"Skipping None item at index {idx}")
            raise IndexError("Invalid item in dataset")
        return item

    def __getattr__(self, name):
        return getattr(self.dataset, name)

# Create Data Loaders
def create_data_loaders(train_df, val_df, test_df, batch_size=32):
    """
    Create DataLoaders for train, validation, and test datasets.

    Args:
        train_df, val_df, test_df (pd.DataFrame): DataFrames for each split.
        batch_size (int): Batch size for DataLoaders.

    Returns:
        tuple: (train_loader, val_loader, test_loader, num_classes)
    """
    # Define transforms
    train_transform = AlbumentationsTransform(get_transforms("train"))
    val_test_transform = AlbumentationsTransform(get_transforms("val"))

    # Create datasets
    try:
        train_dataset = DataFrameDataset(train_df, transform=train_transform)
        val_dataset = DataFrameDataset(val_df, transform=val_test_transform)
        test_dataset = DataFrameDataset(test_df, transform=val_test_transform)
    except Exception as e:
        logger.error(f"Error creating datasets: {e}")
        raise

    # Wrap datasets
    train_dataset_custom = CustomDataset(train_dataset)
    val_dataset_custom = CustomDataset(val_dataset)
    test_dataset_custom = CustomDataset(test_dataset)

    num_classes = len(train_dataset.classes)

    # Create DataLoaders
    train_loader = DataLoader(
        train_dataset_custom,
        batch_size=batch_size,
        shuffle=True,
        num_workers=0,  # Set to 0 for Windows compatibility
        pin_memory=True,
        drop_last=True  # Drop incomplete batches to avoid issues
    )
    val_loader = DataLoader(
        val_dataset_custom,
        batch_size=batch_size,
        shuffle=False,
        num_workers=0,
        pin_memory=True
    )
    test_loader = DataLoader(
        test_dataset_custom,
        batch_size=batch_size,
        shuffle=False,
        num_workers=0,
        pin_memory=True
    )

    # Dataset summary
    logger.info(f"Number of classes: {num_classes}")
    logger.info(f"Classes: {train_dataset.classes}")
    logger.info(f"Training dataset size: {len(train_loader.dataset)}")
    logger.info(f"Validation dataset size: {len(val_loader.dataset)}")
    logger.info(f"Test dataset size: {len(test_loader.dataset)}")
    logger.info(f"Batch size: {batch_size}")

    return train_loader, val_loader, test_loader, num_classes

# Compute class weights for imbalanced dataset
def compute_class_weights(train_df):
    """
    Compute class weights for training based on class distribution.

    Args:
        train_df (pd.DataFrame): Training DataFrame with 'label' column.

    Returns:
        torch.Tensor: Class weights for loss function.
    """
    if train_df.empty:
        logger.error("Training DataFrame is empty, cannot compute class weights.")
        return None
    labels = train_df['label'].values
    unique_labels = np.unique(labels)
    class_weights = compute_class_weight('balanced', classes=unique_labels, y=labels)
    class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)
    logger.info(f"Class weights: {class_weights}")
    return class_weights


# root_dir = r"O:\O drive\AI\my project\medical image projects\breast_termo\archive\BCD_Dataset"
# save_dir = r"O:\O drive\AI\my project\medical image projects\breast_termo\plots"

# Create DataFrames
# train_df, val_df, test_df, class_to_idx = create_dataframes(root_dir)

# # Balance training DataFrame
# train_df_balanced = resample_train_df(train_df, class_to_idx, save_dir)

# Create DataLoaders
batch_size = 64
train_loader, val_loader, test_loader, num_classes = create_data_loaders(
    train_df_balanced, val_df, test_df, batch_size
)

# Compute class weights
class_weights = compute_class_weights(train_df_balanced)

# Memory optimization
torch.cuda.empty_cache()
gc.collect()

2025-05-30 00:12:00,659 - INFO - Using device: cuda
2025-05-30 00:12:00,820 - INFO - Filtered out 0 invalid rows
2025-05-30 00:12:00,821 - INFO - Dataset size after filtering: 210
2025-05-30 00:12:00,822 - INFO - Classes: ['Sick', 'Unknown_class', 'normal']
2025-05-30 00:12:00,823 - INFO - Class to index: {'Sick': 0, 'Unknown_class': 1, 'normal': 2}
2025-05-30 00:12:00,826 - INFO - Class distribution:
class_name
Sick             70
normal           70
Unknown_class    70
Name: count, dtype: int64
2025-05-30 00:12:00,854 - INFO - Filtered out 0 invalid rows
2025-05-30 00:12:00,854 - INFO - Dataset size after filtering: 37
2025-05-30 00:12:00,854 - INFO - Classes: ['Sick', 'Unknown_class', 'normal']
2025-05-30 00:12:00,854 - INFO - Class to index: {'Sick': 0, 'Unknown_class': 1, 'normal': 2}
2025-05-30 00:12:00,854 - INFO - Class distribution:
class_name
Unknown_class    15
Sick             14
normal            8
Name: count, dtype: int64
2025-05-30 00:12:00,900 - INFO - Filtered out 0 i

30

In [6]:
import torch
import time
import logging

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Select CUDA if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
logger.info(f"Using device: {device}")

# Check batch loading time
for i, (inputs, labels) in enumerate(train_loader):
    start_time = time.time()

    # Skip invalid batches
    if inputs is None or labels is None:
        logger.warning(f"Skipping empty batch {i+1}")
        continue

    try:
        # Validate inputs
        if not isinstance(inputs, torch.Tensor):
            logger.error(f"Batch {i+1} inputs is not a tensor: {type(inputs)}")
            continue
        inputs = inputs.to(device)

        # Handle labels (tensor or tuple)
        if isinstance(labels, tuple):
            labels1, labels2, lam = labels
            if not isinstance(labels1, torch.Tensor) or not isinstance(labels2, torch.Tensor):
                logger.error(f"Batch {i+1} labels tuple contains non-tensor: {type(labels1)}, {type(labels2)}")
                continue
            labels = (labels1.to(device), labels2.to(device), lam)
        else:
            if not isinstance(labels, torch.Tensor):
                logger.error(f"Batch {i+1} labels is not a tensor: {type(labels)}")
                continue
            labels = labels.to(device)

        batch_time = time.time() - start_time
        logger.info(f"✅ Batch {i+1} Loaded in {batch_time:.4f} sec")
        logger.info(f"Batch {i+1} inputs shape: {inputs.shape}")
        logger.info(f"Batch {i+1} labels: {labels}")

    except Exception as e:
        logger.error(f"Error processing batch {i+1}: {e}")
        continue

    if i == 3:
        break

# Memory cleanup
torch.cuda.empty_cache()

2025-05-30 00:12:01,501 - INFO - Using device: cuda
2025-05-30 00:12:01,938 - INFO - ✅ Batch 1 Loaded in 0.0000 sec
2025-05-30 00:12:01,938 - INFO - Batch 1 inputs shape: torch.Size([64, 3, 224, 224])
2025-05-30 00:12:01,953 - INFO - Batch 1 labels: tensor([0, 0, 1, 0, 1, 0, 1, 1, 2, 1, 1, 2, 0, 1, 1, 0, 0, 2, 0, 0, 0, 0, 0, 2,
        1, 0, 2, 1, 0, 1, 0, 2, 2, 0, 2, 1, 2, 2, 2, 2, 1, 2, 1, 2, 1, 0, 1, 1,
        2, 1, 0, 2, 2, 1, 1, 0, 2, 2, 0, 0, 0, 0, 2, 0], device='cuda:0')
2025-05-30 00:12:02,208 - INFO - ✅ Batch 2 Loaded in 0.0000 sec
2025-05-30 00:12:02,208 - INFO - Batch 2 inputs shape: torch.Size([64, 3, 224, 224])
2025-05-30 00:12:02,223 - INFO - Batch 2 labels: tensor([2, 1, 1, 1, 0, 1, 0, 2, 1, 2, 2, 1, 1, 1, 2, 1, 2, 2, 2, 2, 1, 1, 0, 0,
        1, 2, 1, 2, 2, 2, 1, 0, 1, 0, 1, 1, 2, 1, 0, 1, 1, 1, 1, 0, 2, 2, 0, 2,
        0, 2, 2, 0, 0, 1, 1, 2, 1, 1, 2, 0, 2, 0, 2, 0], device='cuda:0')
2025-05-30 00:12:02,463 - INFO - ✅ Batch 3 Loaded in 0.0000 sec
2025-05-30 00:12:02,

In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.amp import GradScaler, autocast  # Updated AMP import
from torchvision import models
from torchvision.models import (
    ResNeXt50_32X4D_Weights, DenseNet201_Weights, EfficientNet_B0_Weights,
    ResNet18_Weights, ResNet50_Weights, ResNeXt101_32X8D_Weights
)
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score
import os
import logging

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Early Stopping
class EarlyStopping:
    def __init__(self, patience=10, verbose=False, delta=0, save_dir=None):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_loss = float('inf')
        self.best_epoch = 0
        self.early_stop = False
        self.delta = delta
        self.save_dir = save_dir

    def __call__(self, val_loss, epoch, model_weights, model_name_prefix):
        if val_loss < self.best_loss - self.delta:
            self.best_loss = val_loss
            self.best_epoch = epoch
            self.counter = 0
            self.save_best_weights(model_weights, model_name_prefix)
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
                if self.verbose:
                    logger.info(f"Early stopping triggered after {self.counter} epochs of no improvement.")

    def save_best_weights(self, model_weights, model_name_prefix):
        os.makedirs(self.save_dir, exist_ok=True)
        model_name = os.path.join(self.save_dir, f"{model_name_prefix}_epoch_{self.best_epoch + 1}.pth")
        torch.save(model_weights, model_name)
        if self.verbose:
            logger.info(f"✅ Best model weights saved to {model_name}")

# Custom Classifier
class CustomClassifier(nn.Module):
    def __init__(self, in_features, num_classes):
        super(CustomClassifier, self).__init__()
        self.fc1 = nn.Linear(in_features, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, num_classes)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x

# Supervised Contrastive Loss
class SupConLoss(nn.Module):
    def __init__(self, temperature=0.07):
        super(SupConLoss, self).__init__()
        self.temperature = temperature

    def forward(self, features, labels):
        device = features.device
        batch_size = features.size(0)
        if batch_size < 2:
            return torch.tensor(0.0, device=device)

        features = torch.nn.functional.normalize(features, dim=1)
        similarity_matrix = torch.matmul(features, features.T) / self.temperature
        labels = labels.contiguous().view(-1, 1)
        mask = torch.eq(labels, labels.T).float().to(device)
        exp_sim = torch.exp(similarity_matrix)
        pos_mask = mask - torch.eye(batch_size, device=device)
        pos_sum = (exp_sim * pos_mask).sum(dim=1, keepdim=True)
        neg_sum = (exp_sim * (1 - pos_mask)).sum(dim=1, keepdim=True)
        pos_sum = torch.where(pos_sum == 0, torch.ones_like(pos_sum), pos_sum)
        loss = -torch.log(pos_sum / (pos_sum + neg_sum + 1e-6))
        loss = loss.mean()
        return loss

# Contrastive Model
class ContrastiveModel(nn.Module):
    def __init__(self, base_model, num_classes, feature_dim):
        super(ContrastiveModel, self).__init__()
        self.num_classes = num_classes
        self.feature_dim = feature_dim
        self.backbone = base_model

        if hasattr(self.backbone, 'fc'):
            self.classifier_attr = 'fc'
        elif hasattr(self.backbone, 'classifier'):
            self.classifier_attr = 'classifier'
        else:
            raise ValueError("Backbone has neither 'fc' nor 'classifier' attribute")

        self.feature_extractor = nn.Sequential(*list(self.backbone.children())[:-1])
        self.projection_head = nn.Sequential(
            nn.Linear(self.feature_dim, 512),
            nn.ReLU(),
            nn.Linear(512, 128)
        )

    def forward(self, x):
        features = self.feature_extractor(x)
        features = features.view(features.size(0), -1)
        classifier = getattr(self.backbone, self.classifier_attr)
        logits = classifier(features)
        proj = self.projection_head(features)
        return logits, proj

# Get Model
def get_model(model_name, num_classes, device):
    logger.info(f"Loading model: {model_name} with {num_classes} classes on {device}")

    model_configs = {
        "resnext50": (models.resnext50_32x4d, ResNeXt50_32X4D_Weights.DEFAULT, 2048),
        "densenet201": (models.densenet201, DenseNet201_Weights.DEFAULT, 1920),
        "efficientnet_b0": (models.efficientnet_b0, EfficientNet_B0_Weights.DEFAULT, 1280),
        "resnet18": (models.resnet18, ResNet18_Weights.DEFAULT, 512),
        "resnet50": (models.resnet50, ResNet50_Weights.DEFAULT, 2048),
        "resnext101": (models.resnext101_32x8d, ResNeXt101_32X8D_Weights.DEFAULT, 2048)
    }

    if model_name not in model_configs:
        raise ValueError(f"Model {model_name} not supported.")

    model_fn, weights, in_features = model_configs[model_name]
    base_model = model_fn(weights=weights)

    for param in base_model.parameters():
        param.requires_grad = True

    if hasattr(base_model, 'fc'):
        classifier_attr = 'fc'
    elif hasattr(base_model, 'classifier'):
        classifier_attr = 'classifier'
    else:
        raise ValueError("Cannot find classifier layer")

    custom_classifier = CustomClassifier(in_features, num_classes)
    setattr(base_model, classifier_attr, custom_classifier)

    try:
        model = ContrastiveModel(base_model, num_classes, feature_dim=in_features)
    except Exception as e:
        logger.error(f"Failed to initialize ContrastiveModel: {e}")
        raise

    logger.info(f"Model {model_name} loaded with {in_features} input features to classifier")
    return model.to(device)

# Compute Metrics
def compute_metrics(model, data_loader, device, epoch, split_name, save_dir, class_names):
    model.eval()
    all_labels = []
    all_preds = []
    all_probs = []

    with torch.no_grad():
        for inputs, labels in data_loader:
            if inputs is None or labels is None:
                logger.debug(f"Skipping invalid {split_name} batch")
                continue
            inputs, labels = inputs.to(device), labels.to(device)
            with autocast('cuda'):  # Updated AMP
                logits, _ = model(inputs)
            probs = torch.nn.functional.softmax(logits, dim=1)
            _, preds = torch.max(logits, 1)
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())

    cm = confusion_matrix(all_labels, all_preds)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
    plt.title(f'{split_name.capitalize()} Confusion Matrix - Epoch {epoch}')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.tight_layout()
    os.makedirs(save_dir, exist_ok=True)
    plt.savefig(os.path.join(save_dir, f"{split_name}_confusion_matrix_epoch_{epoch}.png"))
    plt.close()

    report = classification_report(all_labels, all_preds, target_names=class_names, digits=4)
    logger.info(f"\n{split_name.capitalize()} Classification Report - Epoch {epoch}:\n{report}")
    with open(os.path.join(save_dir, f"{split_name}_classification_report_epoch_{epoch}.txt"), 'w') as f:
        f.write(report)

    try:
        roc_auc = roc_auc_score(all_labels, all_probs, multi_class='ovr')
        logger.info(f"{split_name.capitalize()} ROC AUC: {roc_auc:.4f}")
        with open(os.path.join(save_dir, f"{split_name}_roc_auc_epoch_{epoch}.txt"), 'w') as f:
            f.write(f"ROC AUC: {roc_auc:.4f}")
    except ValueError as e:
        logger.warning(f"ROC AUC computation failed for {split_name}: {e}")

# Plot Loss and Accuracy
def plot_loss_accuracy(train_losses, valid_losses, train_accuracies, valid_accuracies, save_dir):
    os.makedirs(save_dir, exist_ok=True)
    epochs = range(1, len(train_losses) + 1)

    plt.figure(figsize=(10, 5))
    plt.plot(epochs, train_losses, 'b-', label='Training Loss')
    plt.plot(epochs, valid_losses, 'r-', label='Validation Loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(os.path.join(save_dir, "loss_curve.png"))
    plt.close()

    plt.figure(figsize=(10, 5))
    plt.plot(epochs, train_accuracies, 'b-', label='Training Accuracy')
    plt.plot(epochs, valid_accuracies, 'r-', label='Validation Accuracy')
    plt.title('Training and Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(os.path.join(save_dir, "accuracy_curve.png"))
    plt.close()

# Train and Validate
def train_and_validate(model, train_loader, val_loader, optimizer, scheduler, 
                       model_name_prefix, class_weights, epochs=25, device=None, 
                       early_stopping=None, save_dir=r"O:\O drive\AI\my project\medical image projects\breast_termo\plots", 
                       accum_steps=4, class_names=['normal', 'Sick', 'Unknown_class']):
    os.makedirs(save_dir, exist_ok=True)

    use_amp = device.type == 'cuda'
    logger.info(f"Using mixed precision training: {use_amp}")

    ce_criterion = nn.CrossEntropyLoss(weight=class_weights)
    supcon_criterion = SupConLoss(temperature=0.07)
    lambda_supcon = 0.3

    train_losses = []
    valid_losses = []
    train_accuracies = []
    valid_accuracies = []
    scaler = GradScaler('cuda') if use_amp else None  # Updated AMP

    for epoch in range(epochs):
        logger.info(f'Epoch {epoch + 1}/{epochs}')
        logger.info('-' * 50)

        model.train()
        running_loss = 0.0
        correct_preds = 0
        total_preds = 0
        optimizer.zero_grad()

        for batch_idx, (inputs, labels) in enumerate(train_loader):
            if inputs is None or labels is None:
                logger.debug("Skipping invalid training batch")
                continue

            inputs, labels = inputs.to(device), labels.to(device)

            if use_amp:
                with autocast('cuda', dtype=torch.float16):  # Updated AMP
                    logits, proj = model(inputs)
                    ce_loss = ce_criterion(logits, labels)
                    supcon_loss = supcon_criterion(proj, labels)
                    total_loss = (ce_loss + lambda_supcon * supcon_loss) / accum_steps
            else:
                logits, proj = model(inputs)
                ce_loss = ce_criterion(logits, labels)
                supcon_loss = supcon_criterion(proj, labels)
                total_loss = (ce_loss + lambda_supcon * supcon_loss) / accum_steps

            if use_amp:
                scaler.scale(total_loss).backward()
                if (batch_idx + 1) % accum_steps == 0 or (batch_idx + 1) == len(train_loader):
                    scaler.step(optimizer)
                    scaler.update()
                    optimizer.zero_grad()
            else:
                total_loss.backward()
                if (batch_idx + 1) % accum_steps == 0 or (batch_idx + 1) == len(train_loader):
                    optimizer.step()
                    optimizer.zero_grad()

            running_loss += total_loss.item() * inputs.size(0) * accum_steps
            _, preds = torch.max(logits, 1)
            correct_preds += (preds == labels).sum().item()
            total_preds += labels.size(0)

        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_acc = correct_preds / total_preds
        train_losses.append(epoch_loss)
        train_accuracies.append(epoch_acc)
        logger.info(f'Training Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}')

        model.eval()
        running_loss = 0.0
        correct_preds = 0
        total_preds = 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                if inputs is None or labels is None:
                    logger.debug("Skipping invalid validation batch")
                    continue

                inputs, labels = inputs.to(device), labels.to(device)
                if use_amp:
                    with autocast('cuda', dtype=torch.float16):  # Updated AMP
                        logits, _ = model(inputs)
                        loss = ce_criterion(logits, labels)
                else:
                    logits, _ = model(inputs)
                    loss = ce_criterion(logits, labels)
                running_loss += loss.item() * inputs.size(0)
                _, preds = torch.max(logits, 1)
                correct_preds += (preds == labels).sum().item()
                total_preds += labels.size(0)

        epoch_loss = running_loss / len(val_loader.dataset)
        epoch_acc = correct_preds / total_preds
        valid_losses.append(epoch_loss)
        valid_accuracies.append(epoch_acc)
        logger.info(f'Validation Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}')

        scheduler.step(epoch_loss)

        if early_stopping:
            early_stopping(epoch_loss, epoch, model.state_dict(), model_name_prefix)
            if early_stopping.early_stop:
                logger.info(f"🚨 Early stopping triggered at epoch {epoch+1}.")
                break

        if epoch % 5 == 0 or (early_stopping and early_stopping.early_stop):
            compute_metrics(model, val_loader, device, epoch + 1, "val", save_dir, class_names)

        torch.cuda.empty_cache()

    if early_stopping and early_stopping.best_epoch is not None:
        logger.info(f"Loading best model weights from epoch {early_stopping.best_epoch + 1}")
        best_model_path = os.path.join(save_dir, f"{model_name_prefix}_epoch_{early_stopping.best_epoch + 1}.pth")
        model.load_state_dict(torch.load(best_model_path))
    else:
        logger.info("No early stopping triggered. Keeping final epoch weights.")

    plot_loss_accuracy(train_losses, valid_losses, train_accuracies, valid_accuracies, save_dir)
    return model



# resnext50 Model

In [12]:

class_weights = compute_class_weights(train_df_balanced)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_name = "resnext50"
model = get_model(model_name, num_classes, device)
optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-2)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)
early_stopping = EarlyStopping(patience=10, verbose=True, save_dir=save_dir)

trained_model = train_and_validate(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    optimizer=optimizer,
    scheduler=scheduler,
    model_name_prefix=model_name,
    class_weights=class_weights,
    epochs=25,
    device=device,
    early_stopping=early_stopping,
    save_dir=save_dir,
    accum_steps=4,
    class_names=['normal', 'Sick', 'Unknown_class']
)

compute_metrics(trained_model, test_loader, device, epoch=early_stopping.best_epoch + 1 or 25, 
                split_name="test", save_dir=save_dir, class_names=['normal', 'Sick', 'Unknown_class'])

2025-05-30 00:15:22,101 - INFO - Class weights: tensor([1., 1., 1.], device='cuda:0')
2025-05-30 00:15:22,101 - INFO - Loading model: resnext50 with 3 classes on cuda
2025-05-30 00:15:22,615 - INFO - Model resnext50 loaded with 2048 input features to classifier
2025-05-30 00:15:22,740 - INFO - Using mixed precision training: True
2025-05-30 00:15:22,742 - INFO - Epoch 1/25
2025-05-30 00:15:22,742 - INFO - --------------------------------------------------
2025-05-30 00:15:26,283 - INFO - Training Loss: 1.7972, Accuracy: 0.3438
2025-05-30 00:15:26,484 - INFO - Validation Loss: 1.1114, Accuracy: 0.2162
2025-05-30 00:15:26,742 - INFO - ✅ Best model weights saved to O:\O drive\AI\my project\medical image projects\breast_termo\plots\resnext50_epoch_1.pth
c:\Users\alira\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `ze

# Resnet18 Model

In [13]:

class_weights = compute_class_weights(train_df_balanced)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_name = "resnet18"
model = get_model(model_name, num_classes, device)
optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-2)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)
early_stopping = EarlyStopping(patience=10, verbose=True, save_dir=save_dir)

trained_model = train_and_validate(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    optimizer=optimizer,
    scheduler=scheduler,
    model_name_prefix=model_name,
    class_weights=class_weights,
    epochs=25,
    device=device,
    early_stopping=early_stopping,
    save_dir=save_dir,
    accum_steps=4,
    class_names=['normal', 'Sick', 'Unknown_class']
)

compute_metrics(trained_model, test_loader, device, epoch=early_stopping.best_epoch + 1 or 25, 
                split_name="test", save_dir=save_dir, class_names=['normal', 'Sick', 'Unknown_class'])

2025-05-30 00:17:15,116 - INFO - Class weights: tensor([1., 1., 1.], device='cuda:0')
2025-05-30 00:17:15,117 - INFO - Loading model: resnet18 with 3 classes on cuda
2025-05-30 00:17:15,531 - INFO - Model resnet18 loaded with 512 input features to classifier
2025-05-30 00:17:15,574 - INFO - Using mixed precision training: True
2025-05-30 00:17:15,574 - INFO - Epoch 1/25
2025-05-30 00:17:15,582 - INFO - --------------------------------------------------
2025-05-30 00:17:17,002 - INFO - Training Loss: 1.4099, Accuracy: 0.3958
2025-05-30 00:17:17,140 - INFO - Validation Loss: 1.0496, Accuracy: 0.3784
2025-05-30 00:17:17,216 - INFO - ✅ Best model weights saved to O:\O drive\AI\my project\medical image projects\breast_termo\plots\resnet18_epoch_1.pth
c:\Users\alira\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_d

# resnext101 Model

In [14]:

class_weights = compute_class_weights(train_df_balanced)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_name = "resnext101"
model = get_model(model_name, num_classes, device)
optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-2)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)
early_stopping = EarlyStopping(patience=10, verbose=True, save_dir=save_dir)

trained_model = train_and_validate(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    optimizer=optimizer,
    scheduler=scheduler,
    model_name_prefix=model_name,
    class_weights=class_weights,
    epochs=25,
    device=device,
    early_stopping=early_stopping,
    save_dir=save_dir,
    accum_steps=4,
    class_names=['normal', 'Sick', 'Unknown_class']
)

compute_metrics(trained_model, test_loader, device, epoch=early_stopping.best_epoch + 1 or 25, 
                split_name="test", save_dir=save_dir, class_names=['normal', 'Sick', 'Unknown_class'])

2025-05-30 00:18:39,536 - INFO - Class weights: tensor([1., 1., 1.], device='cuda:0')
2025-05-30 00:18:39,536 - INFO - Loading model: resnext101 with 3 classes on cuda
2025-05-30 00:18:41,632 - INFO - Model resnext101 loaded with 2048 input features to classifier
2025-05-30 00:18:41,978 - INFO - Using mixed precision training: True
2025-05-30 00:18:41,979 - INFO - Epoch 1/25
2025-05-30 00:18:41,979 - INFO - --------------------------------------------------
2025-05-30 00:19:11,116 - INFO - Training Loss: 1.6300, Accuracy: 0.2917
2025-05-30 00:19:12,799 - INFO - Validation Loss: 1.0905, Accuracy: 0.3784
2025-05-30 00:19:13,467 - INFO - ✅ Best model weights saved to O:\O drive\AI\my project\medical image projects\breast_termo\plots\resnext101_epoch_1.pth
c:\Users\alira\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use 

# resnet50 Model

In [15]:

class_weights = compute_class_weights(train_df_balanced)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_name = "resnet50"
model = get_model(model_name, num_classes, device)
optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-2)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)
early_stopping = EarlyStopping(patience=10, verbose=True, save_dir=save_dir)

trained_model = train_and_validate(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    optimizer=optimizer,
    scheduler=scheduler,
    model_name_prefix=model_name,
    class_weights=class_weights,
    epochs=25,
    device=device,
    early_stopping=early_stopping,
    save_dir=save_dir,
    accum_steps=4,
    class_names=['normal', 'Sick', 'Unknown_class']
)

compute_metrics(trained_model, test_loader, device, epoch=early_stopping.best_epoch + 1 or 25, 
                split_name="test", save_dir=save_dir, class_names=['normal', 'Sick', 'Unknown_class'])

2025-05-30 00:32:23,976 - INFO - Class weights: tensor([1., 1., 1.], device='cuda:0')
2025-05-30 00:32:23,983 - INFO - Loading model: resnet50 with 3 classes on cuda
2025-05-30 00:32:24,685 - INFO - Model resnet50 loaded with 2048 input features to classifier
2025-05-30 00:32:24,791 - INFO - Using mixed precision training: True
2025-05-30 00:32:24,791 - INFO - Epoch 1/25
2025-05-30 00:32:24,791 - INFO - --------------------------------------------------
2025-05-30 00:32:27,044 - INFO - Training Loss: 1.5849, Accuracy: 0.3542
2025-05-30 00:32:27,192 - INFO - Validation Loss: 1.0835, Accuracy: 0.3784
2025-05-30 00:32:27,365 - INFO - ✅ Best model weights saved to O:\O drive\AI\my project\medical image projects\breast_termo\plots\resnet50_epoch_1.pth
c:\Users\alira\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_

# efficientnet Model

In [17]:

class_weights = compute_class_weights(train_df_balanced)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_name = "efficientnet_b0"
model = get_model(model_name, num_classes, device)
optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-2)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)
early_stopping = EarlyStopping(patience=10, verbose=True, save_dir=save_dir)

trained_model = train_and_validate(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    optimizer=optimizer,
    scheduler=scheduler,
    model_name_prefix=model_name,
    class_weights=class_weights,
    epochs=25,
    device=device,
    early_stopping=early_stopping,
    save_dir=save_dir,
    accum_steps=4,
    class_names=['normal', 'Sick', 'Unknown_class']
)

compute_metrics(trained_model, test_loader, device, epoch=early_stopping.best_epoch + 1 or 25, 
                split_name="test", save_dir=save_dir, class_names=['normal', 'Sick', 'Unknown_class'])

2025-05-30 00:34:16,438 - INFO - Class weights: tensor([1., 1., 1.], device='cuda:0')
2025-05-30 00:34:16,438 - INFO - Loading model: efficientnet_b0 with 3 classes on cuda
2025-05-30 00:34:16,716 - INFO - Model efficientnet_b0 loaded with 1280 input features to classifier
2025-05-30 00:34:16,761 - INFO - Using mixed precision training: True
2025-05-30 00:34:16,761 - INFO - Epoch 1/25
2025-05-30 00:34:16,761 - INFO - --------------------------------------------------
2025-05-30 00:34:29,577 - INFO - Training Loss: 2.2120, Accuracy: 0.3177
2025-05-30 00:34:30,705 - INFO - Validation Loss: 1.0942, Accuracy: 0.3514
2025-05-30 00:34:30,829 - INFO - ✅ Best model weights saved to O:\O drive\AI\my project\medical image projects\breast_termo\plots\efficientnet_b0_epoch_1.pth
c:\Users\alira\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicte